In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from math import nan
import glob
import dask
import datetime
from dateutil.relativedelta import relativedelta
from functools import partial
import pandas as pd
import sys
import isla_interp_utils as isla_interp
dask.config.set(**{'array.slicing.split_large_chunks': True})

In [2]:
import importlib
importlib.reload(isla_interp)

<module 'isla_interp_utils' from '/glade/u/home/islas/python/NCAR_CW3E_SMYLE/DATA_SORT/U_1000_10/isla_interp_utils.py'>

In [3]:
def fixtime(dat):
    diff = np.array(dat.time_bnds.isel(nbnd=1)) - np.array(dat.time_bnds.isel(nbnd=0))
    diff = diff/2.
    newtime = np.array(dat.time_bnds.isel(nbnd=0)) + diff
    dat['time'] = newtime
    return dat

### Set up the information

In [4]:
ystart=1970 # start year of hindcasts
yend=2020 # end year of hindcasts
nyears = yend - ystart + 1
initmon = 11
initmonstr=str(initmon).zfill(2)
nmems = 20 # number of hindcast members
memstr = [str(i).zfill(3) for i in np.arange(1,nmems+1,1)] # generating member strings
topdir="/glade/campaign/cesm/development/espwg/SMYLE-CW3E-L83/timeseries/"

In [5]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client

cluster = PBSCluster(
    cores = 1,
    memory = '30GB',
    processes = 1,
    queue = 'casper',
    local_directory='$TMPDIR',
    resource_spec='select=1:ncpus=1:mem=30GB',
    project='P04010022',
    walltime='12:00:00',
    interface='ext')

# scale up
cluster.scale(24)

# change your urls to the dask dashboard so that you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

In [6]:
cluster

Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/islas/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.173:35129,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/islas/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


### Set up preprocessor

In [7]:
### pre-processor to ensure all initialization dates have the same time axis
def preprocessor(ds):
    time = fixtime(ds)
    datestart = pd.to_datetime("1970-"+str(ds.isel(time=0).time.dt.month.values).zfill(2)+"-"+str(ds.isel(time=0).time.dt.day.values).zfill(2), format="%Y-%m-%d")
    time = [ datestart + relativedelta(months=int(i)) for i in np.arange(0,ds.time.size,1) ]
    ds['time'] = time
    return ds

### Read in the geopotential height

In [8]:
phis = xr.open_dataset("/glade/campaign/cesm/collections/CESM2-LE/atm/proc/tseries/month_1/PHIS/"
     +"b.e21.BHISTcmip6.f09_g17.LE2-1001.001.cam.h0.PHIS.185001-185912.nc").isel(time=0).load()

### Pressure levels for output

In [9]:
newlevs=np.array([1000., 975., 950., 925., 900., 875., 850., 825., 800., 775., 750., 700., 650., 600., 550., 450., 
                          400., 350., 300., 250., 225., 175., 150., 125., 100., 70., 50., 30., 20., 10.])*100.

### L83

In [12]:
topdir="/glade/campaign/cesm/development/espwg/SMYLE-CW3E-L83/timeseries/"
expname="b.e21.BSMYLE-CW3E-L83.f09_g17."
outpath="/glade/campaign/cgd/cas/islas/python_savs/NCAR_CW3E_SMYLE/DATA_SORT/U_1000_10/L83/"

for iyear in np.arange(ystart,yend+1,1):
    print(iyear)
    ufiles = sorted([  glob.glob(topdir+expname+str(iyear)+'-'+str(initmon)+'.'+str(imem).zfill(3)+"/atm/proc/tseries/month_1/*.cam.h0.U.*.nc")[0] 
                for imem in np.arange(1,nmems+1,1) ])
    psfiles = sorted([  glob.glob(topdir+expname+str(iyear)+'-'+str(initmon)+'.'+str(imem).zfill(3)+"/atm/proc/tseries/month_1/*.cam.h0.PS.*.nc")[0] 
                for imem in np.arange(1,nmems+1,1) ])

    allulev = []
    for imem in np.arange(0,len(ufiles),1):
        u = xr.open_mfdataset(ufiles[imem],
                          parallel=True, data_vars=['U'], coords='minimal', compat='override', preprocess = partial(preprocessor))
        ps = xr.open_mfdataset(psfiles[imem],
                           parallel=True, data_vars=['PS'], coords='minimal', compat='override', preprocess = partial(preprocessor))
        u_plev = isla_interp.interp_hybrid_to_pressure(
            u.U, ps.PS, u.hyam, u.hybm, p0=1e5, new_levels=newlevs, method='log',
            lev_dim='lev', extrapolate=False, variable='other', t_bot=u.isel(lev=u.lev.size-1), phi_sfc=phis)
        u_plev = u_plev.load()
        allulev.append(u_plev)
    allulev = xr.concat(allulev, dim='M')
    allulev = allulev.rename('U')
    allulev.to_netcdf(outpath+'U_1000_10_L83_init'+initmonstr+'_'+str(iyear)+'.nc')

1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


### L32

In [13]:
topdir="/glade/campaign/cesm/development/espwg/SMYLE-CW3E/timeseries/"
expname="b.e21.BSMYLE-CW3E.f09_g17."
outpath="/glade/campaign/cgd/cas/islas/python_savs/NCAR_CW3E_SMYLE/DATA_SORT/U_1000_10/L32/"

for iyear in np.arange(ystart,yend+1,1):
    print(iyear)
    ufiles = sorted([  glob.glob(topdir+expname+str(iyear)+'-'+str(initmon)+'.'+str(imem).zfill(3)+"/atm/proc/tseries/month_1/*.cam.h0.U.*.nc")[0] 
                for imem in np.arange(1,nmems+1,1) ])
    psfiles = sorted([  glob.glob(topdir+expname+str(iyear)+'-'+str(initmon)+'.'+str(imem).zfill(3)+"/atm/proc/tseries/month_1/*.cam.h0.PS.*.nc")[0] 
                for imem in np.arange(1,nmems+1,1) ])

    allulev = []
    for imem in np.arange(0,len(ufiles),1):
        u = xr.open_mfdataset(ufiles[imem],
                          parallel=True, data_vars=['U'], coords='minimal', compat='override', preprocess = partial(preprocessor))
        ps = xr.open_mfdataset(psfiles[imem],
                           parallel=True, data_vars=['PS'], coords='minimal', compat='override', preprocess = partial(preprocessor))
        u_plev = isla_interp.interp_hybrid_to_pressure(
            u.U, ps.PS, u.hyam, u.hybm, p0=1e5, new_levels=newlevs, method='log',
            lev_dim='lev', extrapolate=False, variable='other', t_bot=u.isel(lev=u.lev.size-1), phi_sfc=phis)
        u_plev = u_plev.load()
        allulev.append(u_plev)
    allulev = xr.concat(allulev, dim='M')
    allulev = allulev.rename('U')
    allulev.to_netcdf(outpath+'U_1000_10_L32_init'+initmonstr+'_'+str(iyear)+'.nc')

1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [14]:
cluster.close()